In [1]:
# Objects for defining the Hamiltonian
from PySW import *
import sympy as sp

In [ ]:
omega_q = RDSymbol('omega_q', real=True, positive=True)
omega_a = RDSymbol('omega_a', real=True, positive=True)

g_p = RDSymbol('g_p', real=False, positive=True, order=1)
omega_p = RDSymbol('omega_p', real=True, positive=True)

Spin = RDBasis('Spin', 2)
s0, sx, sy, sz = Spin.basis

a = BosonOp('a')
ad = Dagger(a)



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
Eff_Frame = EffectiveFrame(H=H0 + HB, V=HE + Hso, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│   a    │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




In [ ]:
Eff_Frame.solve(max_order=5, full_diagonalization=False)
H_eff = Eff_Frame.get_H()
H2_corrections = Eff_Frame.H_corrections

Solving for each order: 100%|██████████| 5/5 [00:21<00:00,  4.26s/it]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
H2 = H2_corrections[2]
sp.expand_complex(H2).trigsimp()

-(\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2 + \tilde{\alpha}**2)/(hbar*omega0)

In [ ]:
H3 = H2_corrections[3]
display_dict({k: sp.expand_complex(v).trigsimp() for k,v in group_by_operators(H3).items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
display_dict({k: sp.expand_complex(v).trigsimp() for k,v in group_by_operators(H2_corrections[4].subs(ad,0).subs(a,0)).items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
get_order(group_by_operators(H2_corrections[4].subs(ad,0).subs(a,0))[sy])

(3, ('other', None))